In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://cloud.r-project.org/b

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReview").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://jpritchardaws-bucket.s3.us-east-2.amazonaws.com/Vine_Table.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Vine_Table.csv"), sep=",", header=True)

# Show DataFrame
df.show()



+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1P1G5KZ05H6RD|          5|            0|          0|   N|                Y|
|R106N066IUN8ZV|          5|            0|          0|   N|                Y|
| R7WTAA1S5O7D9|          4|            0|          0|   N|                Y|
|R32HFMVWLYOYJK|          5|            0|          0|   N|                Y|
| RWT3H6HBVAL6G|          3|            0|          0|   N|                N|
|R1S3T3GWUGQTW7|          5|            0|          0|   N|                Y|
|R3R0QYHA800REE|          4|            0|          0|   N|                Y|
|R1FR0EQPHPW5NM|          4|            1|          2|   N|                N|
| RGORN81H45NI7|          5|            1|          1|   N|                Y|
|R1CNYN4ABNOJSU|          5|            0|          0|   N|     

In [18]:
# Step 1 filter total votes >= 20
filtered_df = df.filter("total_votes>=20").show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RN8GRZ45FA27G|          5|           30|         31|   N|                N|
|R3DHJNHOZTXZ6C|          1|            0|         20|   N|                N|
|R2BVILHJN8OCJB|          5|           29|         31|   N|                N|
|R1L88H90CK9ID5|          1|            1|         37|   N|                Y|
| RJQ36XR04YC0D|          5|           21|         24|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [31]:

df = df.withColumn("helpful_per", df["helpful_votes"] / df["total_votes"]) 
df.show()


+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_per|
+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|R1P1G5KZ05H6RD|          5|            0|          0|   N|                Y|       null|
|R106N066IUN8ZV|          5|            0|          0|   N|                Y|       null|
| R7WTAA1S5O7D9|          4|            0|          0|   N|                Y|       null|
|R32HFMVWLYOYJK|          5|            0|          0|   N|                Y|       null|
| RWT3H6HBVAL6G|          3|            0|          0|   N|                N|       null|
|R1S3T3GWUGQTW7|          5|            0|          0|   N|                Y|       null|
|R3R0QYHA800REE|          4|            0|          0|   N|                Y|       null|
|R1FR0EQPHPW5NM|          4|            1|          2|   N|                N|        0.5|
| RGORN81H

In [46]:
# Data Filtered > 20 total votes and Helpful % >=50%
filtered_df = df.filter("total_votes>=20")
filterTwo_df = filtered_df.filter("helpful_per>=.50").show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_per|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RN8GRZ45FA27G|          5|           30|         31|   N|                N| 0.967741935483871|
|R2BVILHJN8OCJB|          5|           29|         31|   N|                N|0.9354838709677419|
| RJQ36XR04YC0D|          5|           21|         24|   N|                N|             0.875|
|R250VXGBVFWTUV|          4|           17|         20|   N|                N|              0.85|
|R16CPMMKI4ZPZG|          5|           22|         23|   N|                N|0.9565217391304348|
|R1QDLAE82GPWWG|          4|           30|         32|   N|                N|            0.9375|
| ROVJULNDT4MPI|          5|           20|         20|   N|                N|               1.0|
|R3EHTX262RLEKZ|          5|  

In [54]:
# Vine Review
filterYes_df = df.filter(df["vine"] == "Y").filter("total_votes>=20").filter("helpful_per>=.5").show()

+---------+-----------+-------------+-----------+----+-----------------+-----------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_per|
+---------+-----------+-------------+-----------+----+-----------------+-----------+
+---------+-----------+-------------+-----------+----+-----------------+-----------+



In [55]:
# No Vine Review
filterNo_df = df.filter(df["vine"] == "N").filter("total_votes>=20").filter("helpful_per>=.5").show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_per|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RN8GRZ45FA27G|          5|           30|         31|   N|                N| 0.967741935483871|
|R2BVILHJN8OCJB|          5|           29|         31|   N|                N|0.9354838709677419|
| RJQ36XR04YC0D|          5|           21|         24|   N|                N|             0.875|
|R250VXGBVFWTUV|          4|           17|         20|   N|                N|              0.85|
|R16CPMMKI4ZPZG|          5|           22|         23|   N|                N|0.9565217391304348|
|R1QDLAE82GPWWG|          4|           30|         32|   N|                N|            0.9375|
| ROVJULNDT4MPI|          5|           20|         20|   N|                N|               1.0|
|R3EHTX262RLEKZ|          5|  

In [68]:
# Total Reviews
df.select("star_rating").count()


380604

In [72]:
# Five Star Reviews
df.select("star_rating").filter(df["star_rating"] == "5").count()

227500

In [74]:
# % of Vine Paid Reviews (Vine == Y)
df.select("star_rating").filter(df["star_rating"] == "5").filter(df["vine"] == "Y").count() / df.select("star_rating").filter(df["star_rating"] == "5").count()

0.0

In [75]:
# % of Vine unpaid reviews
df.select("star_rating").filter(df["star_rating"] == "5").filter(df["vine"] == "N").count() / df.select("star_rating").filter(df["star_rating"] == "5").count()

1.0